<a href="https://colab.research.google.com/github/ch00226855/CMP414765Fall2022/blob/main/Week12_AnalyzingTexts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 12
# Analyzing Texts

This notebook classifies movie reviews as positive or negative using the text of the review.

We'll use the [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) that contains the text of 50,000 movie reviews from the Internet Movie Database. These reviews are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are balanced, meaning they contain an equal number of positive and negative reviews.

**Please turn on GPU computing from the menu.**

In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Hub version: ", hub.__version__)
print(tf.config.experimental.list_physical_devices("GPU"))

## Download the dataset

In [ ]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

## Explore the Data

In [ ]:
?train_data

In [ ]:
# Turn the training set into an iterator
iterator = iter(train_data.batch(10))

In [ ]:
# Extract the first batch of 10 reviews
train_examples_batch, train_labels_batch = next(iterator) # The next() function returns the next item of an iterator

In [ ]:
# Print a review
print(train_examples_batch[5].numpy())

In [ ]:
# Display the labels of the first 10 reviews
train_labels_batch

## Building the Model
Machine learning models take vectors (arrays of numbers) as input. When working with text, the first thing we must do is come up with a strategy to convert strings to numbers (or to "vectorize" the text) before feeding it to the model. 

### Attempt 1: One-Hot Encoding

The **one-hot encoding** method converts each string to a vector that contains a single non-zero component (thus it is called "one-hot"). It is a commonly used method to convert a categorical variable to a vector.

<img src="https://i.imgur.com/mtimFxh.png" width="400">

However, one-hot encoding is a poor idea for vectorizing texts because:
- The size of the converted vector equals to the size of the vocabulary, which is very large.
- It does not preserve relationships between words. For example, "no" and "not" have similar meanings, but their one-hot vectors are entirely different.
- It is not clear how one can combine all words from a sentence to form a single vector.

<img src="https://miro.medium.com/max/1400/0*QMGjp-fPYpPaE3eK" width="400">





### Attempt 2: The TF-IDF Approach

- **Term Frequency (TF)**: How many times does each word appears?

- **Document Frequency (DF)**: How many documents contains this word?

- **TF-IDF metric**: TF / DF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
email1 = "how are you are you you"
email2 = "hello how you you are hello"
temp_data = np.array([email1, email2])
count = CountVectorizer()
vectors = count.fit_transform(temp_data)
print(vectors)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
vectors = tfidf.fit_transform(vectors)
print(vectors)

### Attempt 3: Use An Existing Word Embedding

For this example we will use a pre-trained text embedding model from TensorFlow Hub called `gnews-swivel-20dim`, which represents text with a vector of length 20. This embedding model is trained with Swivel matrix decomposition method on 130 GB texts from Google News.

In [ ]:
# train_examples_batch.shape
vec1 = hub_layer(["no"]).numpy()
print(vec1)

In [ ]:
vec2 = hub_layer(["not"]).numpy()
print(vec2)

In [ ]:
# Calculate the cosine similarity of these vectors.
cosine = vec1.dot(vec2.T) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
print(cosine)

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

In [ ]:
train_examples_batch[:3]

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# the fit() methods returns a collection of intermediate results, which can be useful
# to evaluate the model
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

## Evaluate the model

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

In [ ]:
# How about my own reviews?
my_review = np.array(["This movie is the worst action movie I have ever watched in my entire life.",
                      "I really enjoyed the plot, but the lead actor didn't portray his character well.",
                      "It is the most visually stunning movie in the series. The acting is outstanding too.",
                      "I really like that everyone in this movie makes it crystal clear that they don't care the quality at all.",
                      "There is nothing about the movie that I don't like. I wish everyone else just stop making movies since no moive can be better than this one."])
model(my_review).numpy()

In [ ]:
# Extract 20 reviews from the test set
reviews, labels = next(iter(test_data.batch(20)))
predictions = model(reviews).numpy()

In [ ]:
labels.numpy()

In [ ]:
(predictions > 0).astype(int).reshape(-1)

### Create Our Own Word Embedding

<a href="https://www.tensorflow.org/text/guide/word_embeddings">TensorFlow tutorial on work embedding</a>